First import the modules we need:

In [1]:
import trajectory, trajectory.post, trajectory.aero, csv
import numpy as np

To run an aerodynamic simulation, we must have already run a trajectory simulation and saved the output to a .JSON. We will now import that JSON:

In [2]:
imported_data = trajectory.from_json("output.json")

We need to recreate the Rocket object that was used to generate output.json. This is because output.json only contains trajectory data, but no details on the actual launch vehicle.

In [3]:
'''Import motor data - copied from Joe Hunt's simulation'''
with open('novus_sim_6.1/motor_out.csv') as csvfile:
    motor_out = csv.reader(csvfile)

    (motor_time_data, prop_mass_data, cham_pres_data,
     throat_data, gamma_data, nozzle_efficiency_data,
     exit_pres_data, area_ratio_data, vmass_data, lden_data, lmass_data, fuel_mass_data) = [], [], [], [], [], [], [], [], [], [], [], []

    next(motor_out)
    for row in motor_out:
        motor_time_data.append(float(row[0]))
        prop_mass_data.append(float(row[1]))
        cham_pres_data.append(float(row[2]))
        throat_data.append(float(row[3]))
        gamma_data.append(float(row[4]))
        nozzle_efficiency_data.append(float(row[5]))
        exit_pres_data.append(float(row[6]))
        area_ratio_data.append(float(row[7]))
        vmass_data.append(float(row[8]))
        lden_data.append(float(row[9]))
        lmass_data.append(float(row[10]))
        fuel_mass_data.append(float(row[11]))
        
        #This is a bit inefficient given that these are constants, (we only need to record them once):
        DENSITY_FUEL = float(row[12])
        DIA_FUEL = float(row[13])
        LENGTH_PORT = float(row[14])

'''Rocket parameters'''
dry_mass = 60                               # kg
rocket_length = 6.529                       # m
rocket_radius = 98.5e-3                     # m
rocket_wall_thickness = 1e-2                # m - This is just needed for the mass model
pos_tank_bottom = 4.456                     # m - Distance between the nose tip and the bottom of the nitrous tank
pos_solidfuel_bottom = 4.856+LENGTH_PORT    # m - Distance between the nose tip and bottom of the solid fuel grain 
ref_area = 0.0305128422                     # m^2 - Reference area for aerodynamic coefficients

'''Set up aerodynamic properties'''
#Get approximate values for the rotational damping coefficients
c_damp_pitch = trajectory.aero.pitch_damping_coefficient(rocket_length, rocket_radius, fin_number = 4, area_per_fin = 0.07369928)
c_damp_roll = 0

#Import drag coefficients from RASAero II
aerodynamic_coefficients = trajectory.aero.RASAeroData("data/Martlet4RasAeroII.CSV", ref_area, c_damp_pitch, c_damp_roll)

'''Set up the mass model'''
liquid_fuel = trajectory.LiquidFuel(lden_data, lmass_data, rocket_radius, pos_tank_bottom, motor_time_data)
solid_fuel = trajectory.SolidFuel(fuel_mass_data, DENSITY_FUEL, DIA_FUEL/2, LENGTH_PORT, pos_solidfuel_bottom, motor_time_data)
dry_mass_model = trajectory.HollowCylinder(rocket_radius, rocket_radius - rocket_wall_thickness, rocket_length, dry_mass)

mass_model = trajectory.HybridMassModel(rocket_length, solid_fuel, liquid_fuel, vmass_data, 
                                        dry_mass_model.mass, dry_mass_model.ixx(), dry_mass_model.iyy(), dry_mass_model.izz(), 
                                        dry_cog = rocket_length/2)

'''Create the other objects needed to initialise the Rocket object'''
pulsar = trajectory.Motor(motor_time_data, 
                          prop_mass_data, 
                          cham_pres_data, 
                          throat_data, 
                          gamma_data, 
                          nozzle_efficiency_data, 
                          exit_pres_data, 
                          area_ratio_data)

launch_site = trajectory.LaunchSite(rail_length=10, 
                                    rail_yaw=0, 
                                    rail_pitch=0, 
                                    alt=0, 
                                    longi=0, 
                                    lat=0, 
                                    wind=[4.94975,4.94975,0])

parachute = trajectory.Parachute(main_s = 13.9,
                                 main_c_d = 0.78,
                                 drogue_s = 1.13,
                                 drogue_c_d = 0.78,
                                 main_alt = 1000,
                                 attatch_distance = 0)

"""Create the Rocket object"""
martlet4 = trajectory.Rocket(mass_model, pulsar, aerodynamic_coefficients, launch_site, h=0.05, variable=True, alt_poll_interval=1, parachute=parachute)


Now we can start actually setting up an aerodynamic heating simulation. First we must define the nosecone shape, by creating a TangentOgive object:

In [4]:
tangent_ogive = trajectory.post.TangentOgive(xprime = 73.7e-2, yprime = (19.7e-2)/2)

'xprime' is the length of the nosecone, and 'yprime' is the radius. See Fig 2. of https://arc.aiaa.org/doi/pdf/10.2514/3.62081 for a diagram.

We can now set up the aerodynamic heating simulation, and run a step:

In [ ]:
analysis = trajectory.post.HeatTransfer(tangent_ogive, imported_data, martlet4)
analysis.step()

Subsonic flow, skipping step number 0


In [ ]:
analysis.run(iterations = 300)

Subsonic flow, skipping step number 1
Subsonic flow, skipping step number 2
Subsonic flow, skipping step number 3
Subsonic flow, skipping step number 4
Subsonic flow, skipping step number 5
Subsonic flow, skipping step number 6
Subsonic flow, skipping step number 7
Subsonic flow, skipping step number 8
Subsonic flow, skipping step number 9
Subsonic flow, skipping step number 10
Subsonic flow, skipping step number 11
Subsonic flow, skipping step number 12
Subsonic flow, skipping step number 13
Subsonic flow, skipping step number 14
Subsonic flow, skipping step number 15
Subsonic flow, skipping step number 16
Subsonic flow, skipping step number 17
Subsonic flow, skipping step number 18
Subsonic flow, skipping step number 19
Subsonic flow, skipping step number 20
Subsonic flow, skipping step number 21
Subsonic flow, skipping step number 22
Subsonic flow, skipping step number 23
Subsonic flow, skipping step number 24
Subsonic flow, skipping step number 25
Subsonic flow, skipping step numbe

 D:\Anaconda3\lib\site-packages\thermo\viscosity.py:826: RuntimeWarning:invalid value encountered in double_scalars


i=100 qxq0 = 0.00040325876242874105 alt = 1371.6184645770118
i=100 qxq0 = 0.0008690638952588758 alt = 1371.6184645770118
i=100 qxq0 = 0.001438138375979636 alt = 1371.6184645770118
i=100 qxq0 = 0.0020658336525868127 alt = 1371.6184645770118
i=100 qxq0 = 0.0027017541949957797 alt = 1371.6184645770118
i=100 qxq0 = 0.003296863600807438 alt = 1371.6184645770118
i=100 qxq0 = 0.0038074644626971276 alt = 1371.6184645770118
i=100 qxq0 = 0.00419771105084465 alt = 1371.6184645770118
i=100 qxq0 = 0.004441071763219218 alt = 1371.6184645770118
i=100 qxq0 = 0.004520942841589519 alt = 1371.6184645770118
i=101 qxq0 = 0.0 alt = 1441.512792344205
i=101 qxq0 = 0.0003941311602455505 alt = 1441.512792344205
i=101 qxq0 = 0.0008491206597004778 alt = 1441.512792344205
i=101 qxq0 = 0.0014054379015209341 alt = 1441.512792344205
i=101 qxq0 = 0.002019469153472979 alt = 1441.512792344205
i=101 qxq0 = 0.002641941837920862 alt = 1441.512792344205
i=101 qxq0 = 0.0032248588633336118 alt = 1441.512792344205
i=101 qxq0 =

i=131 qxq0 = 0.0013772832429232522 alt = 5212.799834063277
i=131 qxq0 = 0.0013739023714206251 alt = 5212.799834063277
i=132 qxq0 = 0.0 alt = 5301.118579796515
i=132 qxq0 = 0.00014858312080066817 alt = 5301.118579796515
i=132 qxq0 = 0.0003108585563391693 alt = 5301.118579796515
i=132 qxq0 = 0.0005003744829316696 alt = 5301.118579796515
i=132 qxq0 = 0.0007002249816007494 alt = 5301.118579796515
i=132 qxq0 = 0.0008932304045098883 alt = 5301.118579796515
i=132 qxq0 = 0.0010642171376854964 alt = 5301.118579796515
i=132 qxq0 = 0.00120102555450766 alt = 5301.118579796515
i=132 qxq0 = 0.001294948480211846 alt = 5301.118579796515
i=132 qxq0 = 0.0013408017636695228 alt = 5301.118579796515
i=132 qxq0 = 0.0013367335710046092 alt = 5301.118579796515
i=133 qxq0 = 0.0 alt = 5390.136102735996
i=133 qxq0 = 0.0001452757286273725 alt = 5390.136102735996
i=133 qxq0 = 0.0003037335941934105 alt = 5390.136102735996
i=133 qxq0 = 0.0004885775364336127 alt = 5390.136102735996
i=133 qxq0 = 0.0006832625571712051 

i=144 qxq0 = 0.0012814482428067117 alt = 5473.420647241175
i=145 qxq0 = 0.0 alt = 5481.824056189507
i=145 qxq0 = 0.0001436172181284144 alt = 5481.824056189507
i=145 qxq0 = 0.0003001801619664863 alt = 5481.824056189507
i=145 qxq0 = 0.00048272419163966325 alt = 5481.824056189507
i=145 qxq0 = 0.0006748867848034871 alt = 5481.824056189507
i=145 qxq0 = 0.0008601050889835008 alt = 5481.824056189507
i=145 qxq0 = 0.0010238079655804397 alt = 5481.824056189507
i=145 qxq0 = 0.0011543739548292773 alt = 5481.824056189507
i=145 qxq0 = 0.0012435375451668924 alt = 5481.824056189507
i=145 qxq0 = 0.00128644061537862 alt = 5481.824056189507
i=145 qxq0 = 0.0012814334295003635 alt = 5481.824056189507
